In [1]:
import os
import pickle

import dotenv
import numpy as np
import openai
import pandas as pd
from tqdm.notebook import tqdm

from lllm import Suspect

dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
rng = np.random.RandomState(42)

In [3]:
from lllm.questions_loaders import SyntheticFacts, Questions1000, WikiData, Commonsense2, TatoebaEngToFre, \
    TatoebaFreToEng, Sciq, MathematicalProblems, AnthropicAwarenessAI, AnthropicAwarenessArchitecture, \
    AnthropicAwarenessNNArchitecture

# all datasets
datasets = [SyntheticFacts(), Questions1000(), WikiData(), Commonsense2(), TatoebaFreToEng(), TatoebaEngToFre(),
            Sciq(), MathematicalProblems(), AnthropicAwarenessAI(), AnthropicAwarenessArchitecture(),
            AnthropicAwarenessNNArchitecture()]

In [4]:
lie_transcripts = []
truth_transcripts = []
for dataset in datasets:
    lie_transcripts.extend(dataset.get_lying_transcripts())
    truth_transcripts.extend(dataset.get_truthful_transcripts())

rng.shuffle(lie_transcripts)
rng.shuffle(truth_transcripts)

len(lie_transcripts), len(truth_transcripts)

(1254, 1328)

In [5]:
def make_prompt(transcript):
    
    return f'Question: {transcript[0]}\nAnswer: {transcript[1]}'

In [6]:
from scipy.special import logsumexp
YES_TOKENS = set(['yes', 'absolutely', 'certainly', 'undoubtedly'])
NO_TOKENS = set(['no', 'never'])
def process_logprobs(logprobs):
    # first, remove all before the *out loud* tokens (if present), for each element in the logprobs
    logprob_differences_list = []
    prob_differences_list = []
    for logprob in logprobs:
        produced_tokens = logprob["tokens"]
        # now loop over the tokens and see if there is a "loud" followed by a "*" token
        found = 0
        for i in range(len(produced_tokens) - 1):
            if "loud" in produced_tokens[i] and "*" in produced_tokens[i + 1]:
                found = i + 2
                break

        # part of the following adapted from Ilan's code
        actual_logprobs = list(logprob["logprobs"][found].values())
        logprob_tokens = list(logprob["logprobs"][found].keys())
        logprob_tokens = [t.lower().strip() for t in logprob_tokens]

        yes_logsum = None
        no_logsum = None
        # now we consider the actual_logprobs at that position, sum those corresponding to "yes" and subtract those corresponding to "no"
        for tok, prob in zip(logprob_tokens, actual_logprobs):
            if tok in YES_TOKENS:
                yes_logsum = np.logaddexp(yes_logsum, prob) if yes_logsum is not None else prob
            elif tok in NO_TOKENS:
                no_logsum = np.logaddexp(no_logsum, prob) if no_logsum is not None else prob

        # if the probability of either yes or no is 0 bound it with the minimum of the probabilities in
        # the list (as those are the top 5) or with 1 - sum of the probabilities in the list (as that bounds
        # the overall prob assigned to the missing tokens).
        if yes_logsum is None:
            yes_logsum = min(min(actual_logprobs), np.log(1 - np.exp(logsumexp(actual_logprobs))))
        if no_logsum is None:
            no_logsum = min(min(actual_logprobs), np.log(1 - np.exp(logsumexp(actual_logprobs))))

        # should I consider the difference between logprobs or between probs?
        logprob_differences_list.append(yes_logsum - no_logsum)
        prob_differences_list.append(np.exp(yes_logsum) - np.exp(no_logsum))

    return logprob_differences_list, prob_differences_list

## Generate logprobs

In [7]:
probes = pd.read_csv("../../data/probes.csv")["probe"].tolist()

In [8]:
model_kwargs = {"temperature": 0.7, "presence_penalty": 0, "frequency_penalty": 0,
                "max_tokens": 128, "top_p": 1, "stop": "\n", "logprobs": 5}

In [9]:
RUN = False

In [10]:
if RUN:
    logprob_differences_lie = []
    for transcript in tqdm(lie_transcripts[:20]):
        prompt = make_prompt(transcript)
        suspect = Suspect(prompt, n_parallel=len(probes), model='text-davinci-003', model_kwargs=model_kwargs)
        answer,logprobs = suspect.get_response(probes, return_logprobs=True)
        logprob_differences_list, prob_differences_list = process_logprobs(logprobs)
        logprob_differences_lie.append(logprob_differences_list)

    logprob_differences_lie = np.array(logprob_differences_lie)

In [11]:
if RUN:
    logprob_differences_true = []
    for transcript in tqdm(truth_transcripts[:20]):
        prompt = make_prompt(transcript)
        suspect = Suspect(prompt, n_parallel=len(probes), model='text-davinci-003', model_kwargs=model_kwargs)
        answer,logprobs = suspect.get_response(probes,return_logprobs=True)
        logprob_differences_list, prob_differences_list = process_logprobs(logprobs)
        logprob_differences_true.append(logprob_differences_list)

    logprob_differences_true = np.array(logprob_differences_true)

In [12]:
if RUN:
    # save to disk
    np.save("../../results/logprob_differences_lie_conversation_only.npy",logprob_differences_lie)
    np.save("../../results/logprob_differences_true_conversation_only.npy",logprob_differences_true)

In [13]:
# load from disk
logprob_differences_lie = np.load("../../results/logprob_differences_lie_conversation_only.npy")
logprob_differences_true = np.load("../../results/logprob_differences_true_conversation_only.npy")

# Test classifiers

In [14]:
with open("../../results/trained_classifiers/logistic_logprobs_classifier_all_probes.pkl", "rb") as f:
    classifier_all = pickle.load(f)
with open("../../results/trained_classifiers/logistic_logprobs_classifier_no_lie.pkl", "rb") as f:
    classifier_no_lie = pickle.load(f)
with open("../../results/trained_classifiers/logistic_logprobs_classifier_lie.pkl", "rb") as f:
    classifier_lie = pickle.load(f)
with open("../../results/trained_classifiers/logistic_logprobs_classifier_knowable.pkl", "rb") as f:
    classifier_knowable = pickle.load(f)

classifiers_logprobs_list = [classifier_all, classifier_no_lie,
                             classifier_lie, classifier_knowable]

# same for binary instead of logprobs:
with open("../../results/trained_classifiers/logistic_binary_classifier_all_probes.pkl", "rb") as f:
    classifier_all_binary = pickle.load(f)
with open("../../results/trained_classifiers/logistic_binary_classifier_no_lie.pkl", "rb") as f:
    classifier_no_lie_binary = pickle.load(f)
with open("../../results/trained_classifiers/logistic_binary_classifier_lie.pkl", "rb") as f:
    classifier_lie_binary = pickle.load(f)
with open("../../results/trained_classifiers/logistic_binary_classifier_knowable.pkl", "rb") as f:
    classifier_knowable_binary = pickle.load(f)

classifiers_binary_list = [classifier_all_binary, classifier_no_lie_binary,
                          classifier_lie_binary, classifier_knowable_binary]

In [15]:
all_indices = np.arange(len(probes))
no_lie_indices = np.load("../../results/probes_groups/no_lie_indices.npy")
lie_indices = np.load("../../results/probes_groups/lie_indices.npy")
knowable_indices = np.load("../../results/probes_groups/knowable_indices.npy")

indeces_list = [all_indices, no_lie_indices, lie_indices, knowable_indices]

all_indices = np.arange(len(probes))
no_lie_indices_binary = np.load("../../results/probes_groups/no_lie_indices.npy")
lie_indices_binary = np.load("../../results/probes_groups/lie_indices.npy")
knowable_indices_binary = np.load("../../results/probes_groups/knowable_indices.npy")

indeces_list_binary = [all_indices, no_lie_indices_binary,
                       lie_indices_binary, knowable_indices_binary]

name_list = ["all_probes", "no_lie", "lie", "knowable"]

In [16]:
logprobs = np.concatenate([logprob_differences_true, logprob_differences_lie])
labels = np.concatenate([np.ones(len(logprob_differences_true)), np.zeros(len(logprob_differences_lie))])

## Logprobs

In [17]:
results_df = pd.DataFrame(columns=["probe_subset", "accuracy", "auc", "y_pred", "y_pred_proba"])

for classifier, indeces, name in zip(classifiers_logprobs_list, indeces_list, name_list):
    accuracy, auc, _, y_pred, y_pred_proba = classifier.evaluate(logprobs[:,indeces], labels, return_ys=True)

    results_df = pd.concat([results_df, pd.DataFrame(
        {"probe_subset": [name], "accuracy": [accuracy], "auc": [auc], "y_pred": [y_pred],
         "y_pred_proba": [y_pred_proba]}) ])
results_df[["probe_subset", "accuracy", "auc"]]

,probe_subset,accuracy,auc
0,all_probes,0.50,0.7100
0,no_lie,0.55,0.6475
0,lie,0.50,0.6200
0,knowable,0.50,0.6025


## Binary

In [18]:
results_df_binary = pd.DataFrame(columns=["probe_subset", "accuracy", "auc", "y_pred", "y_pred_proba"])

for classifier, indeces, name in zip(classifiers_binary_list, indeces_list_binary, name_list):
    accuracy, auc, _, y_pred, y_pred_proba = classifier.evaluate(logprobs[:,indeces]>0, labels, return_ys=True)

    results_df_binary = pd.concat([results_df_binary, pd.DataFrame(
        {"probe_subset": [name], "accuracy": [accuracy], "auc": [auc], "y_pred": [y_pred],
         "y_pred_proba": [y_pred_proba]}) ])
results_df_binary[["probe_subset", "accuracy", "auc"]]

,probe_subset,accuracy,auc
0,all_probes,0.5,0.51500
0,no_lie,0.5,0.46125
0,lie,0.5,0.50250
0,knowable,0.5,0.55000
